In [1]:
import librosa
import librosa.display
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pylab
from scipy import signal
from tqdm.notebook import tqdm
from glob import glob
import gc

import tensorflow as tf
import keras
from keras import layers, losses
from keras.models import Model, load_model
import keras.backend as K
from keras.activations import leaky_relu
from keras.losses import binary_crossentropy, mean_squared_error

from keras.layers import Input,Conv2D, Layer, UpSampling2D, MaxPooling2D, Dense, Flatten, Lambda, GlobalAveragePooling2D, Conv2DTranspose, Reshape, BatchNormalization, Dropout, ELU, ReLU
from tensorflow.keras.utils import plot_model

import os
import pdb

import ast

c:\Users\GP65\anaconda3\envs\py39\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
main_data = pd.read_csv('D:/Projects/Orpheus_ai/DataSet/main_dataframe.csv')

In [3]:
main_data['main_genre'] = main_data['track genres'].apply(lambda a : ast.literal_eval(a)[0])

In [4]:
data = main_data['main_genre'].value_counts().sort_values()[::-1].cumsum()
top_genre = data.head(30).index

In [5]:
## Genre Ranking
genre_names = pd.read_csv('D:/Projects/fma/data/fma_metadata/genres.csv')
genre_names = pd.merge(genre_names,main_data['main_genre'].value_counts(),right_index=True,left_on='genre_id')
genre_names[genre_names['genre_id'].isin(top_genre)].sort_values('main_genre')[::-1]

,genre_id,#tracks,parent,title,top_level,main_genre
14,15,34413,0,Electronic,15,3855
11,12,32923,0,Rock,12,2406
20,21,8389,0,Hip-Hop,21,1957
22,25,9261,12,Punk,12,1401
16,17,12706,0,Folk,17,1155
17,18,5913,1235,Soundtrack,1235,773
9,10,13845,0,Pop,10,653
31,38,38154,0,Experimental,38,647
0,1,8693,38,Avant-Garde,38,559
44,66,5432,12,Indie-Rock,12,556


In [6]:
## 30 class encodings
encodings = pd.get_dummies(main_data[main_data['main_genre'].isin(top_genre)].set_index('track_id')['main_genre'])

## Getting the DataSet

In [7]:
train_paths = glob('D:/Projects/Orpheus_ai/DataSet/Spectrograms/HalfMEL_dbscale/*')[:-100]
test_paths = glob('D:/Projects/Orpheus_ai/DataSet/Spectrograms/HalfMEL_dbscale/*')[-100:]
vqvae = load_model('D:/Projects/Orpheus_ai/DataSet/model_save_logs/Saved Models (Aux)/VQVAE(7K)_basicAux_3e-03recloss',compile=False)

BATCH_SIZE = 4
AUTOTUNE = tf.data.experimental.AUTOTUNE
min_level_db = -80 

In [8]:
def get_code_indices(embeddings, flattened_inputs):
    # Calculate L2-normalized distance between the inputs and the codes.
    similarity = tf.matmul(flattened_inputs, embeddings)
    distances = (
        tf.reduce_sum(flattened_inputs ** 2, axis=1, keepdims=True)
        + tf.reduce_sum(embeddings ** 2, axis=0)
        - 2 * similarity
    )

    # Derive the indices for minimum distances.
    encoding_indices = tf.argmin(distances, axis=1)
    return encoding_indices

def data_loader(path):
    path = bytes.decode(path.numpy())
    spec = tf.cast(np.load(path),tf.float32)
    spec = tf.expand_dims(spec,axis=-1)
    return spec

def normalize(S):
    return np.clip((((S - min_level_db) / -min_level_db)*2.)-1., -1, 1)

def denormalize(S):
    return (((np.clip(S, -1, 1)+1.)/2.) * -min_level_db) + min_level_db

def qunatize(S):
    S = tf.expand_dims(S,axis=0)
    encodings = vqvae.encoder(S)
    ## Flatten the encodings
    original_shape = encodings.shape[1:-1]
    encodings = tf.reshape(encodings,[-1,encodings.shape[-1]])
    ## Get the indices of the encodings
    indices = get_code_indices(vqvae.quantizer.layers[-1].embeddings,encodings)
    ## Reshape indices into encodings shape
    indices = tf.reshape(indices,original_shape)
    return indices,indices


In [25]:
def get_dataset(data_paths):
    dataset = tf.data.Dataset.from_tensor_slices(data_paths)
    dataset = dataset.map(lambda x : tf.py_function(data_loader, [x], [tf.float32]), num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(lambda x : tf.py_function(normalize, [x], [tf.float32]), num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(lambda x : tf.py_function(qunatize, [x], [tf.int64,tf.int64]), num_parallel_calls=AUTOTUNE)    
    dataset = dataset.batch(BATCH_SIZE)
    it = dataset.__iter__() #missing line
    it = it.repeat()
    
    return it


In [26]:
X_train = get_dataset(train_paths)
X_test = get_dataset(test_paths)
# X_test = np.array([np.expand_dims(normalize(np.load(i)),axis=-1) for i in test_paths])

AttributeError: 'OwnedIterator' object has no attribute 'repeat'

## Model Architecture

In [27]:
class PixelConvLayer(layers.Layer):
    def __init__(self, mask_type, **kwargs):
        super().__init__()
        self.mask_type = mask_type
        self.conv = layers.Conv2D(**kwargs)

    def build(self, input_shape):
        # Build the conv2d layer to initialize kernel variables
        self.conv.build(input_shape)
        # Use the initialized kernel to create the mask
        kernel_shape = self.conv.kernel.get_shape()
        self.mask = np.zeros(shape=kernel_shape)
        self.mask[: kernel_shape[0] // 2, ...] = 1.0
        self.mask[kernel_shape[0] // 2, : kernel_shape[1] // 2, ...] = 1.0
        if self.mask_type == "B":
            self.mask[kernel_shape[0] // 2, kernel_shape[1] // 2, ...] = 1.0

    def call(self, inputs):
        self.conv.kernel.assign(self.conv.kernel * self.mask)
        return self.conv(inputs)
    
class ResidualBlock(keras.layers.Layer):
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = keras.layers.Conv2D(
            filters=filters, kernel_size=1, activation="relu"
        )
        self.pixel_conv = PixelConvLayer(
            mask_type="B",
            filters=filters // 2,
            kernel_size=3,
            activation="relu",
            padding="same",
        )
        self.conv2 = keras.layers.Conv2D(
            filters=filters, kernel_size=1, activation="relu"
        )

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pixel_conv(x)
        x = self.conv2(x)
        return keras.layers.add([inputs, x])



In [28]:
num_residual_blocks = 5
num_pixelcnn_layers = 5
pixelcnn_input_shape = (32, 54)
num_embeddings = 2048

In [29]:
pixelcnn_inputs = keras.Input(shape=pixelcnn_input_shape, dtype=tf.int32)
ohe = tf.one_hot(pixelcnn_inputs, num_embeddings)

x = PixelConvLayer(
    mask_type="A", filters=128, kernel_size=7, activation="relu", padding="same"
)(ohe)

for _ in range(num_residual_blocks):
    x = ResidualBlock(filters=128)(x)

for _ in range(num_pixelcnn_layers):
    x = PixelConvLayer(
        mask_type="B",
        filters=128,
        kernel_size=1,
        strides=1,
        activation="relu",
        padding="valid",
    )(x)

out = keras.layers.Conv2D(
    filters=num_embeddings, kernel_size=1, strides=1, padding="valid"
)(x)

pixel_cnn = keras.Model(pixelcnn_inputs, out, name="pixel_cnn")
pixel_cnn.summary()

Model: "pixel_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 54)]          0         
                                                                 
 tf.one_hot_2 (TFOpLambda)   (None, 32, 54, 2048)      0         
                                                                 
 pixel_conv_layer_14 (PixelC  (None, 32, 54, 128)      12845184  
 onvLayer)                                                       
                                                                 
 residual_block_6 (ResidualB  (None, 32, 54, 128)      98624     
 lock)                                                           
                                                                 
 residual_block_7 (ResidualB  (None, 32, 54, 128)      98624     
 lock)                                                           
                                                         

In [30]:
pixel_cnn.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [33]:
pixel_cnn.fit(X_train,
    epochs=30,
    steps_per_epoch=250
)

Epoch 1/30
 31/250 [==>...........................] - ETA: 32s - loss: 7.2872 - accuracy: 0.0077

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} InvalidArgumentError: {{function_node __wrapped__Sub_device_/job:localhost/replica:0/task:0/device:GPU:0}} required broadcastable shapes [Op:Sub]
Traceback (most recent call last):

  File "c:\Users\GP65\anaconda3\envs\py39\lib\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    return func(device, token, args)

  File "c:\Users\GP65\anaconda3\envs\py39\lib\site-packages\tensorflow\python\ops\script_ops.py", line 147, in __call__
    outputs = self._call(device, args)

  File "c:\Users\GP65\anaconda3\envs\py39\lib\site-packages\tensorflow\python\ops\script_ops.py", line 154, in _call
    ret = self._func(*args)

  File "c:\Users\GP65\anaconda3\envs\py39\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwa